In [1]:
import pandas as pd
import sqlite3
import numpy as np

file_path= r"C:\Users\NNG Mathur\Desktop\Blinkit\BlinkIT_Grocery_Data.xlsx"
sheet_name="BlinkIT Grocery Data"

try:
    df_raw = pd.read_excel(file_path, sheet_name=sheet_name)
except FileNotFoundError:
    # Handle the case if the file path or sheet name is wrong.
    print(f"Error: File not found or sheet name '{sheet_name}' is incorrect. Please check the path.")
    exit()

conn=sqlite3.connect(':memory:')
table_name='raw_sales_data'

df_raw.to_sql(table_name,conn,index=False,if_exists='replace')

#1. SQL Cleaning and Standardization (Demonstrates SQL DML skills) 
print("Running SQL data standardization...")
update_fat_content_query_lf = f"""
UPDATE {table_name}
set "Item Fat Content"='Low Fat'
WHERE "Item Fat Content" IN ('low fat', 'LF');
"""
conn.execute(update_fat_content_query_lf)

update_fat_content_query_reg = f"""
UPDATE {table_name}
SET "Item Fat Content" = 'Regular'
WHERE "Item Fat Content" = 'reg';
"""
conn.execute(update_fat_content_query_reg)

print("Running SQL imputation for Item Weight...")
impute_weight_query = f"""
update {table_name} as t1
set "Item Weight"=(
select avg(t2."Item Weight")
from {table_name} as t2
where t2."Item Identifier"=t1."Item Identifier"
)
where t1."Item Weight" is null;
"""

conn.execute(impute_weight_query)

mode_size_query=f"""
select "Outlet Type", "Outlet Size", count(*) as count 
from {table_name}
where "Outlet Size" is not null
group by "Outlet Type", "Outlet Size"
order by "Outlet Type", count desc;
"""
print("\nTop Outlet Sizes by Type (for manual imputation logic):")
print(pd.read_sql(mode_size_query, conn))

#2. Extract the cleaned data into a new Pandas DataFrame   
select_all_query=f"select * from {table_name};"
df_cleaned=pd.read_sql(select_all_query,conn)

conn.close()

print("\n--- SQL Wrangling Complete ---")
print("Cleaned DataFrame Head:")
print(df_cleaned.head())
print("\nUnique Item Fat Content after standardization:", df_cleaned['Item Fat Content'].unique())
print("Missing Item Weight after imputation:", df_cleaned['Item Weight'].isnull().sum())
# Note: You may still have missing Outlet_Size if an Outlet_Type has no recorded size.
print("Missing Outlet Size:", df_cleaned['Outlet Size'].isnull().sum())


Running SQL data standardization...
Running SQL imputation for Item Weight...

Top Outlet Sizes by Type (for manual imputation logic):
         Outlet Type Outlet Size  count
0      Grocery Store       Small    585
1      Grocery Store      Medium    265
2      Grocery Store        High    233
3  Supermarket Type1       Small   2554
4  Supermarket Type1        High   1520
5  Supermarket Type1      Medium   1503
6  Supermarket Type2      Medium    928
7  Supermarket Type3      Medium    935

--- SQL Wrangling Complete ---
Cleaned DataFrame Head:
  Item Fat Content Item Identifier              Item Type  \
0          Regular           FDX32  Fruits and Vegetables   
1          Low Fat           NCB42     Health and Hygiene   
2          Regular           FDR28           Frozen Foods   
3          Regular           FDL50                 Canned   
4          Low Fat           DRI25            Soft Drinks   

   Outlet Establishment Year Outlet Identifier Outlet Location Type  \
0          

In [2]:
# Assuming 'df_cleaned' is the DataFrame resulting from your SQL code

# 1. FINAL IMPUTATION FOR 'Item Weight'
# The remaining 4 missing values belong to Item_Identifiers 
# that only had NULL weights. We'll impute them with the overall median.
median_item_weight = df_cleaned['Item Weight'].median()
df_cleaned['Item Weight'].fillna(median_item_weight, inplace=True)
print(f"Final Item Weight Imputation: Missing values are now {df_cleaned['Item Weight'].isnull().sum()}.")

# 2. IMPUTATION FOR 'Outlet Size'
# We'll use the modal relationship observed in your SQL output to impute missing sizes.
# Based on your SQL output:
# - Outlet Type 'Grocery Store' modal size is 'Small'.
# - Supermarket Type 1, 2, 3 all had 'Medium' as a high-frequency size, 
#   but since Outlet Type OUT019 (a Grocery Store) had missing size and 'Grocery Store' is mostly 'Small',
#   we'll impute 'Outlet Size' = 'Small' for missing values.

# Note: In the original data, Outlet_Identifier 'OUT010' and 'OUT019' (both Grocery Store) have missing 'Outlet Size' (or are null/small, depending on the Excel mapping)
df_cleaned['Outlet Size'].fillna('Small', inplace=True)
print(f"Outlet Size Imputation Complete: Missing values are now {df_cleaned['Outlet Size'].isnull().sum()}.")
print(f"Outlet Sizes after imputation: {df_cleaned['Outlet Size'].unique()}")


# 3. FEATURE ENGINEERING: Calculate Outlet Age
current_year = 2024 # Assuming current year is 2024
df_cleaned['Outlet_Age'] = current_year - df_cleaned['Outlet Establishment Year']
df_cleaned.drop('Outlet Establishment Year', axis=1, inplace=True)


# 4. FEATURE ENGINEERING: Handle zero 'Item Visibility' and Log Transform 'Sales'

# Replace zero visibility with a tiny value to avoid log(0) issues, and then log-transform it
df_cleaned['Item Visibility'] = np.where(
    df_cleaned['Item Visibility'] == 0,
    1e-9,  # Use 1e-9 (a very small number)
    df_cleaned['Item Visibility']
)
df_cleaned['Item_Visibility_Log'] = np.log(df_cleaned['Item Visibility'])

# Log transform the target variable ('Sales') to normalize the distribution
df_cleaned['Sales_Log'] = np.log1p(df_cleaned['Sales']) 
df_cleaned.drop('Sales', axis=1, inplace=True) # Drop the original Sales column

print("\n--- Final Data Prep Summary ---")
print(df_cleaned.head())
print(f"\nNew features created: ['Outlet_Age', 'Item_Visibility_Log', 'Sales_Log']")

Final Item Weight Imputation: Missing values are now 0.
Outlet Size Imputation Complete: Missing values are now 0.
Outlet Sizes after imputation: ['Medium' 'Small' 'High']

--- Final Data Prep Summary ---
  Item Fat Content Item Identifier              Item Type Outlet Identifier  \
0          Regular           FDX32  Fruits and Vegetables            OUT049   
1          Low Fat           NCB42     Health and Hygiene            OUT018   
2          Regular           FDR28           Frozen Foods            OUT046   
3          Regular           FDL50                 Canned            OUT013   
4          Low Fat           DRI25            Soft Drinks            OUT045   

  Outlet Location Type Outlet Size        Outlet Type  Item Visibility  \
0               Tier 1      Medium  Supermarket Type1         0.100014   
1               Tier 3      Medium  Supermarket Type2         0.008596   
2               Tier 1       Small  Supermarket Type1         0.025896   
3               Tier 3  

C:\Users\NNG Mathur\AppData\Local\Temp\ipykernel_16900\3768736371.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_cleaned['Item Weight'].fillna(median_item_weight, inplace=True)
C:\Users\NNG Mathur\AppData\Local\Temp\ipykernel_16900\3768736371.py:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values alway

In [3]:
# --- ADD THIS TO THE END OF YOUR PHASE 1 NOTEBOOK ---

# 1. Define the name of the file to save the cleaned data
output_file_name = 'blinkit_cleaned_data_for_ml.csv'

# 2. Save the final DataFrame to a CSV file.
# We set index=False because the index isn't needed as a column.
df_cleaned.to_csv(output_file_name, index=False)

print(f"\n✅ Cleaned data successfully exported as: {output_file_name}")


✅ Cleaned data successfully exported as: blinkit_cleaned_data_for_ml.csv
